# Ten

This example tries to demonstrate working with uncommon [time signatures](https://en.wikipedia.org/wiki/Time_signature) in wubwub.  Namely, it is in 10/8.

Sequencers in wubwub aren't aware of a time signature, only the number of beats specified.  It's up to the user to provide the content to generate the feel of the desired time signature.  So to create a song in 10/8, we don't need to do anything special (aside from setting the length of the Sequencer to be divisible by 10).

While I would describe this time signature as 10/8, it is sort of encoded as a *very fast* 5/4.  By doubling the BPM (400, rather than 200), we can work with nicer whole numbers (`1, 2, ..., 9, 10`) rather than many halves (`1, 1.5, ..., 5, 5.5`).

With a count this fast, and an irregular time signature, it can be difficult to think past the first measure.  *Patterns* (i.e. `wubwub.Pattern`) can help you work in shorter chunks.  Patterns contain a list of beats and a length; various methods allow you to recreate the same rhythmic pattern at different parts of (or throughout) the song.

This song also shows creating an initial Sequencer (`seq`), copying it (`intro`), and then joining (`final`) to make a beat with multiple sections.  Like Patterns, this technique is helpful for making longer creations.

In [1]:
from pysndfx import AudioEffectsChain
import wubwub as wb
import wubwub.sounds as snd

# load sounds
ORGAN = snd.load('keys.organ')
DRUMS = snd.load('drums.house')
DRUMS2 = snd.load('drums.ukhard')
FX = snd.load('synth.fx')
SYNTH = snd.load('synth.ts_synth')

# initialize the sequencer
end = 40
seq = wb.Sequencer(bpm=400, beats=end)

# create Patterns

#    emphasized and non-emphasized beats
emph = wb.Pattern([1, 4, 7, 9], length=10).until(end)
notemph = wb.Pattern([2, 3, 5, 6, 8, 10], length=10).until(end)

#    all beats
every = emph.merge(notemph)

#    patterns for individual instruments
kickpat = wb.Pattern([1, 4, 6], length=10).until(end)
snarepat = wb.Pattern([7, 10], length=10).until(end)
bleeppat = wb.Pattern([1,2,], length=4).until(end)
synthpat = wb.Pattern([1, 4, 6, 7], length=10).until(end)

#    make a pattern that phases in and out of the 10/8
phase = wb.Pattern([1], length=4).until(end)

#    pattern for a hihat count
count = wb.Pattern([1, 3, 5, 7, 9], length=10).until(end)

# the main bass-y organ
organ = seq.add_sampler(ORGAN['C1'], name='organ', basepitch='C1')
organ.make_notes(every, pitches=[3,2,0,5,3,2,7,3,5,2])

# drums
hat = seq.add_sampler(DRUMS['hat1'], name='hat')
hat.make_notes(emph, volumes=0)
hat.make_notes(notemph, volumes=-5, lengths=.1)
hat.volume -= 10
hat.pan = .5

snare = seq.add_sampler(DRUMS['snare3'], name='snare')
snare.make_notes(snarepat, volumes=[0, -5])
snare.volume -= 5

kick = seq.add_sampler(DRUMS['kick6'], name='kick')
kick.make_notes(kickpat)
kick.volume -= 2

# poor man's side chaining: lower the organ volume when the kick occurs
organ[kick.array_of_beats()] = wb.alter_notes(organ[kick.array_of_beats()], volume=-5)

ride = seq.add_sampler(DRUMS2['ride-hard'], name='ride')
ride.make_notes(phase, lengths=5)
ride.pan = -.6
ride.effects = AudioEffectsChain().highpass(5000)

# little synth FX
bleep = seq.add_sampler(wb.shift_pitch(FX['checkpoint-hit'], 8), name='bleep')
bleep.make_notes(bleeppat)
bleep.effects = AudioEffectsChain().reverb(wet_gain=5)
bleep.volume -= 10

# higher synth
synth = seq.add_sampler(SYNTH['patch001'], name='synth')
synth.make_notes(every, pitches=[12,12,12,0,0,0,12,12,0,0], lengths=.2)
synth.effects = AudioEffectsChain().highpass(1000).reverb()

# copy the original sequence just to keep the instruments
intro = seq.copy(with_notes=False)

# create an intro with just the organ
intro['organ'].add_fromdict(seq['organ'].notedict)
intro['organ'].effects = AudioEffectsChain().lowpass(200)

# add drums to build up to the rest of the beat
intro['hat'].make_notes(count.onmeasure(3, measurelen=10), lengths=.1)
intro['snare'].make_notes([37, 38, 39, 40])

# join the intro and main section, and render
final = wb.join([intro, seq])
final.build(overhang=5)